In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from icecream import ic
from sklearn.preprocessing import LabelEncoder
import time
from itertools import product
from icecream import ic


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

sales_train = pd.read_csv('./data/sales_train.csv')
test = pd.read_csv('./data/test.csv')   # (214200, 3)

# 计算每个商品每个月的销售量，假如某个商品在某个月没有数据，则填充0（即这个月的销售量为0）
sales_by_item_id = sales_train.pivot_table(index=['item_id'], values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
sales_by_item_id.columns = sales_by_item_id.columns.droplevel().map(str)   # 去掉第一层索引
sales_by_item_id.columns.values[0] = 'item_id'
sales_by_item_id = sales_by_item_id.rename_axis(None, axis=1)

# 获取最近6个月销售量为0的数据
# six_zero = sales_by_item_id[(sales_by_item_id['28'] == 0) & (sales_by_item_id['29'] == 0) & (sales_by_item_id['30'] == 0) & (sales_by_item_id['31'] == 0) & (sales_by_item_id['32'] == 0) & (sales_by_item_id['33'] == 0)]
# six_zero_item_id = list(six_zero['item_id'].values)   # item_id列表
# test.loc[test.item_id.isin(six_zero_item_id), 'item_cnt_month'] = 0  # 将test数据中（最近六个月销量为0）的数据月销量设为0，有7812个

# 计算每个商店每个月的销量
sales_by_shop_id = sales_train.pivot_table(index=['shop_id'], values=['item_cnt_day'], aggfunc=np.sum, fill_value=0, columns='date_block_num').reset_index()
sales_by_shop_id.columns = sales_by_shop_id.columns.droplevel().map(str)    # 将两层column转化为一层column,保留下层column
sales_by_shop_id.columns.values[0] = 'shop_id'
sales_by_shop_id = sales_by_shop_id.rename_axis(None, axis=1)   # 将列方向的轴重命名为none

In [2]:
sales_by_shop_id

,shop_id,0,1,2,3,4,5,6,7,8,...,24,25,26,27,28,29,30,31,32,33
0,0,5578,6127,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2947,3364,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1146,488,753,583,553,832,807,875,945,...,987,907,762,859,843,804,785,942,822,727
3,3,767,798,959,647,710,784,721,890,1026,...,977,738,741,740,731,672,535,666,745,613
4,4,2114,2025,2060,285,1211,1464,1378,1713,1536,...,1188,980,978,899,893,793,842,947,732,831
5,5,0,877,1355,1008,1110,1393,1265,1510,1298,...,1404,1101,1109,1054,1012,954,991,1294,1092,1052
6,6,3686,4007,4519,3168,3022,3847,3360,3702,4208,...,2328,2329,1981,1998,1748,1539,1484,1575,1725,1802
7,7,2495,2513,2460,1540,1647,2085,2031,2397,2599,...,2084,1847,1430,1340,1217,1235,1327,1409,1287,1212
8,8,1463,1156,977,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,0,0,0,3186


In [3]:
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = sales_train[sales_train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix.sort_values(cols, inplace=True)  # 排序
sales_train['revenue'] = sales_train['item_price'] * sales_train['item_cnt_day']    # 某一天的销售额

In [4]:
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,02.01.2013,0,59,22154,999.00,1.0,999.00
1,03.01.2013,0,25,2552,899.00,1.0,899.00
2,05.01.2013,0,25,2552,899.00,-1.0,-899.00
3,06.01.2013,0,25,2554,1709.05,1.0,1709.05
4,15.01.2013,0,25,2555,1099.00,1.0,1099.00
...,...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0,299.00
2935845,09.10.2015,33,25,7460,299.00,1.0,299.00
2935846,14.10.2015,33,25,7459,349.00,1.0,349.00
2935847,22.10.2015,33,25,7440,299.00,1.0,299.00


In [5]:
groupby = sales_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day': 'sum'}).reset_index()

In [6]:
groupby

,shop_id,item_id,date_block_num,item_cnt_day
0,0,30,1,31.0
1,0,31,1,11.0
2,0,32,0,6.0
3,0,32,1,10.0
4,0,33,0,3.0
...,...,...,...,...
1609119,59,22164,27,2.0
1609120,59,22164,30,1.0
1609121,59,22167,9,1.0
1609122,59,22167,11,2.0


In [7]:
groupby = groupby.rename(columns={'item_cnt_day': 'item_cnt_month'})
groupby

,shop_id,item_id,date_block_num,item_cnt_month
0,0,30,1,31.0
1,0,31,1,11.0
2,0,32,0,6.0
3,0,32,1,10.0
4,0,33,0,3.0
...,...,...,...,...
1609119,59,22164,27,2.0
1609120,59,22164,30,1.0
1609121,59,22167,9,1.0
1609122,59,22167,11,2.0


In [8]:
matrix

,date_block_num,shop_id,item_id
139255,0,0,19
141495,0,0,27
144968,0,0,28
142661,0,0,29
138947,0,0,32
...,...,...,...
10768834,33,59,22162
10769024,33,59,22163
10769690,33,59,22164
10771216,33,59,22166


In [9]:
matrix = matrix.merge(groupby, on=['date_block_num','shop_id','item_id'], how='left')
matrix['item_cnt_month'] = matrix['item_cnt_month'].fillna(0).clip(0, 20)
matrix['item_cnt_month'] = matrix['item_cnt_month'].astype(np.float16)

In [10]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,19,0.0
1,0,0,27,0.0
2,0,0,28,0.0
3,0,0,29,0.0
4,0,0,32,6.0
...,...,...,...,...
10913845,33,59,22162,0.0
10913846,33,59,22163,0.0
10913847,33,59,22164,0.0
10913848,33,59,22166,0.0


In [11]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [12]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

In [13]:
test

,ID,shop_id,item_id,date_block_num
0,0,5,5037,34
1,1,5,5320,34
2,2,5,5233,34
3,3,5,5232,34
4,4,5,5268,34
...,...,...,...,...
214195,214195,45,18454,34
214196,214196,45,16188,34
214197,214197,45,15757,34
214198,214198,45,19648,34


In [14]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,19,0.0
1,0,0,27,0.0
2,0,0,28,0.0
3,0,0,29,0.0
4,0,0,32,6.0
...,...,...,...,...
10913845,33,59,22162,0.0
10913846,33,59,22163,0.0
10913847,33,59,22164,0.0
10913848,33,59,22166,0.0


In [15]:
matrix = pd.concat([matrix, test[cols]], ignore_index=True, axis=0)
matrix

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,19,0.0
1,0,0,27,0.0
2,0,0,28,0.0
3,0,0,29,0.0
4,0,0,32,6.0
...,...,...,...,...
11128045,34,45,18454,NaN
11128046,34,45,16188,NaN
11128047,34,45,15757,NaN
11128048,34,45,19648,NaN


In [16]:
matrix['item_cnt_month'].fillna(0, inplace=True)

In [17]:
items = pd.read_csv('./data/items.csv')
items

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62


In [18]:
items = items[['item_id', 'item_category_id']]
items

,item_id,item_category_id
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40
...,...,...
22165,22165,31
22166,22166,54
22167,22167,49
22168,22168,62


In [19]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,19,0.0
1,0,0,27,0.0
2,0,0,28,0.0
3,0,0,29,0.0
4,0,0,32,6.0
...,...,...,...,...
11128045,34,45,18454,0.0
11128046,34,45,16188,0.0
11128047,34,45,15757,0.0
11128048,34,45,19648,0.0


In [20]:
matrix = pd.merge(left=matrix, right=items, on='item_id', how='left')  # 合并
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id
0,0,0,19,0.0,40
1,0,0,27,0.0,19
2,0,0,28,0.0,30
3,0,0,29,0.0,23
4,0,0,32,6.0,40
...,...,...,...,...,...
11128045,34,45,18454,0.0,55
11128046,34,45,16188,0.0,64
11128047,34,45,15757,0.0,55
11128048,34,45,19648,0.0,40


In [21]:
le = LabelEncoder()
categories = pd.read_csv('./data/item_categories.csv')
categories

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


In [22]:
categories['split'] = categories['item_category_name'].str.split('-')
categories['type'] = categories['split'].map(lambda x:x[0].strip())
categories['subtype'] = categories['split'].map(lambda x:x[1].strip() if len(x)>1 else x[0].strip())
categories = categories[['item_category_id','type','subtype']]
categories

,item_category_id,type,subtype
0,0,PC,Гарнитуры/Наушники
1,1,Аксессуары,PS2
2,2,Аксессуары,PS3
3,3,Аксессуары,PS4
4,4,Аксессуары,PSP
...,...,...,...
79,79,Служебные,Служебные
80,80,Служебные,Билеты
81,81,Чистые носители (шпиль),Чистые носители (шпиль)
82,82,Чистые носители (штучные),Чистые носители (штучные)


In [23]:
categories['cat_type_code'] = le.fit_transform(categories['type'])
categories['cat_subtype_code'] = le.fit_transform(categories['subtype'])
categories

d:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,item_category_id,type,subtype,cat_type_code,cat_subtype_code
0,0,PC,Гарнитуры/Наушники,0,29
1,1,Аксессуары,PS2,1,9
2,2,Аксессуары,PS3,1,10
3,3,Аксессуары,PS4,1,11
4,4,Аксессуары,PSP,1,13
...,...,...,...,...,...
79,79,Служебные,Служебные,16,54
80,80,Служебные,Билеты,16,25
81,81,Чистые носители (шпиль),Чистые носители (шпиль),17,62
82,82,Чистые носители (штучные),Чистые носители (штучные),18,63


In [24]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id
0,0,0,19,0.0,40
1,0,0,27,0.0,19
2,0,0,28,0.0,30
3,0,0,29,0.0,23
4,0,0,32,6.0,40
...,...,...,...,...,...
11128045,34,45,18454,0.0,55
11128046,34,45,16188,0.0,64
11128047,34,45,15757,0.0,55
11128048,34,45,19648,0.0,40


In [25]:
matrix = pd.merge(left=matrix, right=categories[['item_category_id','cat_type_code','cat_subtype_code']], on='item_category_id', how='left')    # 合并
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,cat_subtype_code
0,0,0,19,0.0,40,11,4
1,0,0,27,0.0,19,5,10
2,0,0,28,0.0,30,8,55
3,0,0,29,0.0,23,5,16
4,0,0,32,6.0,40,11,4
...,...,...,...,...,...,...,...
11128045,34,45,18454,0.0,55,13,2
11128046,34,45,16188,0.0,64,14,42
11128047,34,45,15757,0.0,55,13,2
11128048,34,45,19648,0.0,40,11,4


In [26]:
shops = pd.read_csv('./data/shops.csv')
shops

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [27]:
shops['split']=shops.shop_name.str.split(' ')
shops['shop_city'] = shops['split'].map(lambda x:x[0])
shops

,shop_name,shop_id,split,shop_city
0,"!Якутск Орджоникидзе, 56 фран",0,"[!Якутск, Орджоникидзе,, 56, фран]",!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,"[!Якутск, ТЦ, ""Центральный"", фран]",!Якутск
2,"Адыгея ТЦ ""Мега""",2,"[Адыгея, ТЦ, ""Мега""]",Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,"[Балашиха, ТРК, ""Октябрь-Киномир""]",Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,"[Волжский, ТЦ, ""Волга, Молл""]",Волжский
5,"Вологда ТРЦ ""Мармелад""",5,"[Вологда, ТРЦ, ""Мармелад""]",Вологда
6,"Воронеж (Плехановская, 13)",6,"[Воронеж, (Плехановская,, 13)]",Воронеж
7,"Воронеж ТРЦ ""Максимир""",7,"[Воронеж, ТРЦ, ""Максимир""]",Воронеж
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,"[Воронеж, ТРЦ, Сити-Парк, ""Град""]",Воронеж
9,Выездная Торговля,9,"[Выездная, Торговля]",Выездная


In [28]:
shops['shop_city_code'] = le.fit_transform(shops['shop_city'])
def st(name):
    if 'ТЦ' in name or 'ТРЦ' in name:
        shopt = 'ТЦ'
    elif 'ТК' in name:
        shopt = 'ТК'
    elif 'ТРК' in name:
        shopt = 'ТРК'
    elif 'МТРЦ' in name:
        shopt = 'МТРЦ'
    else:
        shopt = 'UNKNOWN'
    return shopt

In [29]:
shops['shop_type'] = shops['shop_name'].apply(st)
shops

,shop_name,shop_id,split,shop_city,shop_city_code,shop_type
0,"!Якутск Орджоникидзе, 56 фран",0,"[!Якутск, Орджоникидзе,, 56, фран]",!Якутск,0,UNKNOWN
1,"!Якутск ТЦ ""Центральный"" фран",1,"[!Якутск, ТЦ, ""Центральный"", фран]",!Якутск,0,ТЦ
2,"Адыгея ТЦ ""Мега""",2,"[Адыгея, ТЦ, ""Мега""]",Адыгея,1,ТЦ
3,"Балашиха ТРК ""Октябрь-Киномир""",3,"[Балашиха, ТРК, ""Октябрь-Киномир""]",Балашиха,2,ТРК
4,"Волжский ТЦ ""Волга Молл""",4,"[Волжский, ТЦ, ""Волга, Молл""]",Волжский,3,ТЦ
5,"Вологда ТРЦ ""Мармелад""",5,"[Вологда, ТРЦ, ""Мармелад""]",Вологда,4,ТЦ
6,"Воронеж (Плехановская, 13)",6,"[Воронеж, (Плехановская,, 13)]",Воронеж,5,UNKNOWN
7,"Воронеж ТРЦ ""Максимир""",7,"[Воронеж, ТРЦ, ""Максимир""]",Воронеж,5,ТЦ
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,"[Воронеж, ТРЦ, Сити-Парк, ""Град""]",Воронеж,5,ТЦ
9,Выездная Торговля,9,"[Выездная, Торговля]",Выездная,6,UNKNOWN


In [30]:
shops.loc[shops.shop_id == 21, 'shop_type'] = 'МТРЦ'   # 修正

In [31]:
shops['shop_type_code'] = le.fit_transform(shops['shop_type'])
shops

,shop_name,shop_id,split,shop_city,shop_city_code,shop_type,shop_type_code
0,"!Якутск Орджоникидзе, 56 фран",0,"[!Якутск, Орджоникидзе,, 56, фран]",!Якутск,0,UNKNOWN,0
1,"!Якутск ТЦ ""Центральный"" фран",1,"[!Якутск, ТЦ, ""Центральный"", фран]",!Якутск,0,ТЦ,4
2,"Адыгея ТЦ ""Мега""",2,"[Адыгея, ТЦ, ""Мега""]",Адыгея,1,ТЦ,4
3,"Балашиха ТРК ""Октябрь-Киномир""",3,"[Балашиха, ТРК, ""Октябрь-Киномир""]",Балашиха,2,ТРК,3
4,"Волжский ТЦ ""Волга Молл""",4,"[Волжский, ТЦ, ""Волга, Молл""]",Волжский,3,ТЦ,4
5,"Вологда ТРЦ ""Мармелад""",5,"[Вологда, ТРЦ, ""Мармелад""]",Вологда,4,ТЦ,4
6,"Воронеж (Плехановская, 13)",6,"[Воронеж, (Плехановская,, 13)]",Воронеж,5,UNKNOWN,0
7,"Воронеж ТРЦ ""Максимир""",7,"[Воронеж, ТРЦ, ""Максимир""]",Воронеж,5,ТЦ,4
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,"[Воронеж, ТРЦ, Сити-Парк, ""Град""]",Воронеж,5,ТЦ,4
9,Выездная Торговля,9,"[Выездная, Торговля]",Выездная,6,UNKNOWN,0


In [32]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,cat_subtype_code
0,0,0,19,0.0,40,11,4
1,0,0,27,0.0,19,5,10
2,0,0,28,0.0,30,8,55
3,0,0,29,0.0,23,5,16
4,0,0,32,6.0,40,11,4
...,...,...,...,...,...,...,...
11128045,34,45,18454,0.0,55,13,2
11128046,34,45,16188,0.0,64,14,42
11128047,34,45,15757,0.0,55,13,2
11128048,34,45,19648,0.0,40,11,4


In [33]:
matrix = pd.merge(left=matrix, right=shops[['shop_id','shop_city_code','shop_type_code']], on='shop_id', how='left')    # 合并
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,cat_subtype_code,shop_city_code,shop_type_code
0,0,0,19,0.0,40,11,4,0,0
1,0,0,27,0.0,19,5,10,0,0
2,0,0,28,0.0,30,8,55,0,0
3,0,0,29,0.0,23,5,16,0,0
4,0,0,32,6.0,40,11,4,0,0
...,...,...,...,...,...,...,...,...,...
11128045,34,45,18454,0.0,55,13,2,21,4
11128046,34,45,16188,0.0,64,14,42,21,4
11128047,34,45,15757,0.0,55,13,2,21,4
11128048,34,45,19648,0.0,40,11,4,21,4


In [34]:
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['cat_type_code'] = matrix['cat_type_code'].astype(np.int8)
matrix['cat_subtype_code'] = matrix['cat_subtype_code'].astype(np.int8)
matrix['shop_city_code'] = matrix['shop_city_code'].astype(np.int8)
matrix['shop_type_code'] = matrix['shop_type_code'].astype(np.int8)

In [38]:
def lag_features(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id',col+'_lag_'+str(i)]
        shifted['date_block_num'] = shifted['date_block_num'] + i
        df = pd.merge(left=df, right=shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df
matrix = lag_features(matrix, [1,2,3,6,12], 'item_cnt_month')

In [39]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,cat_subtype_code,shop_city_code,shop_type_code,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12
0,0,0,19,0.0,40,11,4,0,0,NaN,NaN,NaN,NaN,NaN
1,0,0,27,0.0,19,5,10,0,0,NaN,NaN,NaN,NaN,NaN
2,0,0,28,0.0,30,8,55,0,0,NaN,NaN,NaN,NaN,NaN
3,0,0,29,0.0,23,5,16,0,0,NaN,NaN,NaN,NaN,NaN
4,0,0,32,6.0,40,11,4,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11128045,34,45,18454,0.0,55,13,2,21,4,1.0,0.0,0.0,0.0,NaN
11128046,34,45,16188,0.0,64,14,42,21,4,0.0,0.0,NaN,NaN,NaN
11128047,34,45,15757,0.0,55,13,2,21,4,0.0,0.0,0.0,0.0,0.0
11128048,34,45,19648,0.0,40,11,4,21,4,0.0,0.0,0.0,0.0,NaN


In [40]:
# 月销量（所有商品）
group = matrix.groupby('date_block_num').agg({'item_cnt_month': 'mean'}).reset_index()
group.columns = ['date_block_num', 'date_avg_item_cnt']
matrix = pd.merge(left=matrix, right=group, on='date_block_num', how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_avg_item_cnt')
matrix.drop('date_avg_item_cnt', axis=1, inplace=True)

In [41]:
# 月销量（每一件商品）
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop('date_item_avg_item_cnt', axis=1, inplace=True)

In [42]:
# 月销量（每个商店 ）
group = matrix.groupby(['date_block_num','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop('date_shop_avg_item_cnt', axis=1, inplace=True)

In [43]:
# 月销量（每个类别）
group = matrix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_avg_item_cnt')
matrix.drop('date_cat_avg_item_cnt', axis=1, inplace=True)

In [44]:
# 月销量（商品类别-商店）
group = matrix.groupby(['date_block_num','item_category_id','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_shop_avg_item_cnt')
matrix.drop('date_cat_shop_avg_item_cnt', axis=1, inplace=True)

In [45]:
# 月销量（商品大类）
group = matrix.groupby(['date_block_num','cat_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_type_avg_item_cnt')
matrix.drop('date_type_avg_item_cnt', axis=1, inplace=True)

In [46]:
# 月销量（商店城市）
group = matrix.groupby(['date_block_num','shop_city_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_city_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_city_avg_item_cnt')
matrix.drop('date_city_avg_item_cnt', axis=1, inplace=True)

In [47]:
# 月销量（商品-商店城市）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_city_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_city_avg_item_cnt')
matrix.drop('date_item_city_avg_item_cnt', axis=1, inplace=True)

In [48]:
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,02.01.2013,0,59,22154,999.00,1.0,999.00
1,03.01.2013,0,25,2552,899.00,1.0,899.00
2,05.01.2013,0,25,2552,899.00,-1.0,-899.00
3,06.01.2013,0,25,2554,1709.05,1.0,1709.05
4,15.01.2013,0,25,2555,1099.00,1.0,1099.00
...,...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0,299.00
2935845,09.10.2015,33,25,7460,299.00,1.0,299.00
2935846,14.10.2015,33,25,7459,349.00,1.0,349.00
2935847,22.10.2015,33,25,7440,299.00,1.0,299.00


In [49]:
# 趋势特征
group = sales_train.groupby('item_id').agg({'item_price': 'mean'})
group.columns = ['item_avg_item_price']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = sales_train.groupby(['date_block_num','item_id']).agg({'item_price': 'mean'})
group.columns = ['date_item_avg_item_price']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'date_item_avg_item_price')
for i in lags:
    matrix['delta_price_lag_'+str(i)] = (matrix['date_item_avg_item_price_lag_' + str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if pd.notnull(row['delta_price_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_price_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)

features_to_drop = ['item_avg_item_price','date_item_avg_item_price']
for i in lags:
    features_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    features_to_drop += ['delta_price_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

In [50]:
# 每个月的天数
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days)
matrix['days'] = matrix['days'].astype(np.int8)

In [51]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,cat_subtype_code,shop_city_code,shop_type_code,item_cnt_month_lag_1,...,date_city_avg_item_cnt_lag_6,date_city_avg_item_cnt_lag_12,date_item_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_2,date_item_city_avg_item_cnt_lag_3,date_item_city_avg_item_cnt_lag_6,date_item_city_avg_item_cnt_lag_12,delta_price_lag,month,days
0,0,0,19,0.0,40,11,4,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,31
1,0,0,27,0.0,19,5,10,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,31
2,0,0,28,0.0,30,8,55,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,31
3,0,0,29,0.0,23,5,16,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,31
4,0,0,32,6.0,40,11,4,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11128045,34,45,18454,0.0,55,13,2,21,4,1.0,...,0.143433,NaN,0.5,0.0,0.0,0.0,NaN,-0.475098,10,30
11128046,34,45,16188,0.0,64,14,42,21,4,0.0,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.081116,10,30
11128047,34,45,15757,0.0,55,13,2,21,4,0.0,...,0.143433,0.182007,0.0,0.5,0.0,0.0,0.0,0.155884,10,30
11128048,34,45,19648,0.0,40,11,4,21,4,0.0,...,0.143433,NaN,0.0,0.0,0.0,0.0,NaN,-0.091736,10,30


In [53]:
matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')

0            0
1            0
2            0
3            0
4            0
            ..
11128045    23
11128046    32
11128047     0
11128048    23
11128049    17
Name: date_block_num, Length: 11128050, dtype: int8

In [54]:
# 开始销量
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

In [55]:
# 因为有12个月的延迟特征（1，2，3，6，12）（1，2，3，4，5，6，12），所以需要删除前12月的数据
matrix = matrix[matrix['date_block_num'] > 11]

In [56]:
columns = matrix.columns
column_null = []
for i in columns:
    if len(matrix[matrix[i].isnull()]) > 0:
        column_null.append(i)
column_null

['item_cnt_month_lag_1',
 'item_cnt_month_lag_2',
 'item_cnt_month_lag_3',
 'item_cnt_month_lag_6',
 'item_cnt_month_lag_12',
 'date_avg_item_cnt_lag_1',
 'date_avg_item_cnt_lag_2',
 'date_avg_item_cnt_lag_3',
 'date_avg_item_cnt_lag_6',
 'date_avg_item_cnt_lag_12',
 'date_item_avg_item_cnt_lag_1',
 'date_item_avg_item_cnt_lag_2',
 'date_item_avg_item_cnt_lag_3',
 'date_item_avg_item_cnt_lag_6',
 'date_item_avg_item_cnt_lag_12',
 'date_shop_avg_item_cnt_lag_1',
 'date_shop_avg_item_cnt_lag_2',
 'date_shop_avg_item_cnt_lag_3',
 'date_shop_avg_item_cnt_lag_6',
 'date_shop_avg_item_cnt_lag_12',
 'date_cat_avg_item_cnt_lag_1',
 'date_cat_avg_item_cnt_lag_2',
 'date_cat_avg_item_cnt_lag_3',
 'date_cat_avg_item_cnt_lag_6',
 'date_cat_avg_item_cnt_lag_12',
 'date_cat_shop_avg_item_cnt_lag_1',
 'date_cat_shop_avg_item_cnt_lag_2',
 'date_cat_shop_avg_item_cnt_lag_3',
 'date_cat_shop_avg_item_cnt_lag_6',
 'date_cat_shop_avg_item_cnt_lag_12',
 'date_type_avg_item_cnt_lag_1',
 'date_type_avg_item_

In [57]:
for i in column_null:
    matrix[i].fillna(0, inplace=True)

d:\program files\python37\lib\site-packages\pandas\core\series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [58]:
columns = matrix.columns
column_null = []
for i in columns:
    if len(matrix[matrix[i].isnull()]) > 0:
        column_null.append(i)
column_null

[]

In [59]:
trainData = matrix[matrix['date_block_num'] < 34]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

validData = matrix[matrix['date_block_num'] == 34]
label_valid = validData['item_cnt_month']
X_valid = validData.drop('item_cnt_month', axis=1)

In [61]:
import lightgbm as lgb
train_data = lgb.Dataset(data=X_train, label=label_train)
valid_data = lgb.Dataset(data=X_valid, label=label_valid)

In [62]:
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 1000,
    'num_leaves': 200,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.01,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}

In [63]:
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data])

d:\program files\python37\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.171824 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9550
[LightGBM] [Info] Number of data points in the train set: 6425094, number of used features: 58
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Info] Start training from score 0.287729
[1]	training's rmse: 1.18208	valid_1's rmse: 0.287704
[2]	training's rmse: 1.17756	valid_1's rmse: 0.287803
[3]	training's rmse: 1.17314	valid_1's rmse: 0.288
[4]	training's rmse: 1.16812	valid_1's rmse: 0.288551
[5]	training's rmse: 1.16392	valid_1's rmse: 0.289066
[6]	training's rmse: 1.15909	valid_1's rmse: 0.28986
[7]	training's rmse: 1.15564	valid_1's rmse: 0.290797
[8]	training's rmse: 1.1508	valid_1's rms

[138]	training's rmse: 0.874901	valid_1's rmse: 0.58904
[139]	training's rmse: 0.873899	valid_1's rmse: 0.591128
[140]	training's rmse: 0.873106	valid_1's rmse: 0.592995
[141]	training's rmse: 0.872541	valid_1's rmse: 0.594177
[142]	training's rmse: 0.871866	valid_1's rmse: 0.595712
[143]	training's rmse: 0.87114	valid_1's rmse: 0.597159
[144]	training's rmse: 0.870239	valid_1's rmse: 0.598976
[145]	training's rmse: 0.869436	valid_1's rmse: 0.600765
[146]	training's rmse: 0.868472	valid_1's rmse: 0.602648
[147]	training's rmse: 0.867798	valid_1's rmse: 0.604334
[148]	training's rmse: 0.867024	valid_1's rmse: 0.605734
[149]	training's rmse: 0.86623	valid_1's rmse: 0.607794
[150]	training's rmse: 0.865246	valid_1's rmse: 0.609679
[151]	training's rmse: 0.864677	valid_1's rmse: 0.611442
[152]	training's rmse: 0.863817	valid_1's rmse: 0.613373
[153]	training's rmse: 0.863265	valid_1's rmse: 0.614584
[154]	training's rmse: 0.862375	valid_1's rmse: 0.616416
[155]	training's rmse: 0.861549	va

[283]	training's rmse: 0.806698	valid_1's rmse: 0.748451
[284]	training's rmse: 0.806556	valid_1's rmse: 0.748851
[285]	training's rmse: 0.806345	valid_1's rmse: 0.749452
[286]	training's rmse: 0.806167	valid_1's rmse: 0.750104
[287]	training's rmse: 0.805897	valid_1's rmse: 0.75058
[288]	training's rmse: 0.805662	valid_1's rmse: 0.750943
[289]	training's rmse: 0.805488	valid_1's rmse: 0.751339
[290]	training's rmse: 0.805324	valid_1's rmse: 0.751634
[291]	training's rmse: 0.804829	valid_1's rmse: 0.752374
[292]	training's rmse: 0.804634	valid_1's rmse: 0.753033
[293]	training's rmse: 0.804385	valid_1's rmse: 0.753481
[294]	training's rmse: 0.804042	valid_1's rmse: 0.754236
[295]	training's rmse: 0.80352	valid_1's rmse: 0.754491
[296]	training's rmse: 0.803351	valid_1's rmse: 0.755418
[297]	training's rmse: 0.803122	valid_1's rmse: 0.756066
[298]	training's rmse: 0.802961	valid_1's rmse: 0.756426
[299]	training's rmse: 0.802783	valid_1's rmse: 0.756904
[300]	training's rmse: 0.802551	v

[428]	training's rmse: 0.778668	valid_1's rmse: 0.803978
[429]	training's rmse: 0.778486	valid_1's rmse: 0.80463
[430]	training's rmse: 0.77832	valid_1's rmse: 0.804776
[431]	training's rmse: 0.778159	valid_1's rmse: 0.805046
[432]	training's rmse: 0.778077	valid_1's rmse: 0.805255
[433]	training's rmse: 0.777963	valid_1's rmse: 0.805344
[434]	training's rmse: 0.777843	valid_1's rmse: 0.805572
[435]	training's rmse: 0.777758	valid_1's rmse: 0.80577
[436]	training's rmse: 0.777685	valid_1's rmse: 0.805973
[437]	training's rmse: 0.777608	valid_1's rmse: 0.806138
[438]	training's rmse: 0.777509	valid_1's rmse: 0.806407
[439]	training's rmse: 0.777424	valid_1's rmse: 0.80648
[440]	training's rmse: 0.777292	valid_1's rmse: 0.806521
[441]	training's rmse: 0.777208	valid_1's rmse: 0.806716
[442]	training's rmse: 0.777126	valid_1's rmse: 0.806966
[443]	training's rmse: 0.776967	valid_1's rmse: 0.807361
[444]	training's rmse: 0.776879	valid_1's rmse: 0.807648
[445]	training's rmse: 0.776705	val

[573]	training's rmse: 0.763464	valid_1's rmse: 0.82721
[574]	training's rmse: 0.763413	valid_1's rmse: 0.827222
[575]	training's rmse: 0.763325	valid_1's rmse: 0.827346
[576]	training's rmse: 0.763243	valid_1's rmse: 0.827522
[577]	training's rmse: 0.76285	valid_1's rmse: 0.827493
[578]	training's rmse: 0.762752	valid_1's rmse: 0.827624
[579]	training's rmse: 0.76262	valid_1's rmse: 0.827839
[580]	training's rmse: 0.762555	valid_1's rmse: 0.827918
[581]	training's rmse: 0.762474	valid_1's rmse: 0.827923
[582]	training's rmse: 0.762346	valid_1's rmse: 0.827994
[583]	training's rmse: 0.762279	valid_1's rmse: 0.828076
[584]	training's rmse: 0.762221	valid_1's rmse: 0.828168
[585]	training's rmse: 0.762099	valid_1's rmse: 0.828161
[586]	training's rmse: 0.762022	valid_1's rmse: 0.828124
[587]	training's rmse: 0.761952	valid_1's rmse: 0.828186
[588]	training's rmse: 0.761847	valid_1's rmse: 0.82847
[589]	training's rmse: 0.761766	valid_1's rmse: 0.828435
[590]	training's rmse: 0.761695	val

[718]	training's rmse: 0.75159	valid_1's rmse: 0.840221
[719]	training's rmse: 0.751529	valid_1's rmse: 0.840404
[720]	training's rmse: 0.75148	valid_1's rmse: 0.84052
[721]	training's rmse: 0.75143	valid_1's rmse: 0.840623
[722]	training's rmse: 0.751377	valid_1's rmse: 0.840683
[723]	training's rmse: 0.751337	valid_1's rmse: 0.840758
[724]	training's rmse: 0.751243	valid_1's rmse: 0.840867
[725]	training's rmse: 0.751199	valid_1's rmse: 0.840978
[726]	training's rmse: 0.751144	valid_1's rmse: 0.84104
[727]	training's rmse: 0.751076	valid_1's rmse: 0.841057
[728]	training's rmse: 0.751029	valid_1's rmse: 0.841529
[729]	training's rmse: 0.75099	valid_1's rmse: 0.841603
[730]	training's rmse: 0.750896	valid_1's rmse: 0.841598
[731]	training's rmse: 0.750816	valid_1's rmse: 0.841709
[732]	training's rmse: 0.750768	valid_1's rmse: 0.841719
[733]	training's rmse: 0.750727	valid_1's rmse: 0.84175
[734]	training's rmse: 0.750661	valid_1's rmse: 0.84181
[735]	training's rmse: 0.750576	valid_1

[863]	training's rmse: 0.742623	valid_1's rmse: 0.848015
[864]	training's rmse: 0.74259	valid_1's rmse: 0.847988
[865]	training's rmse: 0.742539	valid_1's rmse: 0.847987
[866]	training's rmse: 0.742507	valid_1's rmse: 0.848028
[867]	training's rmse: 0.742467	valid_1's rmse: 0.848039
[868]	training's rmse: 0.742416	valid_1's rmse: 0.848001
[869]	training's rmse: 0.742373	valid_1's rmse: 0.847973
[870]	training's rmse: 0.74233	valid_1's rmse: 0.847954
[871]	training's rmse: 0.742299	valid_1's rmse: 0.848003
[872]	training's rmse: 0.742252	valid_1's rmse: 0.848005
[873]	training's rmse: 0.742211	valid_1's rmse: 0.848051
[874]	training's rmse: 0.742162	valid_1's rmse: 0.848039
[875]	training's rmse: 0.742053	valid_1's rmse: 0.848254
[876]	training's rmse: 0.741984	valid_1's rmse: 0.848238
[877]	training's rmse: 0.741915	valid_1's rmse: 0.84855
[878]	training's rmse: 0.741846	valid_1's rmse: 0.848559
[879]	training's rmse: 0.741806	valid_1's rmse: 0.848588
[880]	training's rmse: 0.741768	va

In [64]:
# test数据
testData = matrix[matrix['date_block_num'] == 34]
X_test = testData.drop('item_cnt_month', axis=1)

In [65]:
# 预测&生成文件
y_test = lgb_model.predict(X_test).clip(0, 20)
submission = pd.DataFrame({ 'ID': range(0, 214200), 'item_cnt_month': y_test})
submission.to_csv('./submit/sub3.csv', index=False)